In [6]:
import requests
import smtplib, email
import os
import pygame
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.header import Header
import threading
from bs4 import BeautifulSoup
import time
import datetime

In [7]:
class EmailSender:
    def __init__(self, addr, pwd):
        self.smtp = addr.split("@")[1]
        self.addr = addr
        self.pwd  = pwd

    def send(self, subject, content):
        msg = MIMEMultipart("mixed")
        msg["Subject"] = subject
        msg["From"]    = self.addr
        msg["To"]      = self.addr
        text = MIMEText(content, "html", "utf-8")
        msg.attach(text)
        smtp = smtplib.SMTP()
        smtp.connect(self.smtp)
        smtp.login(self.addr, self.pwd)
        smtp.sendmail(self.addr, self.addr, msg.as_string())
        smtp.quit()
        
class DesktopToaster:
    def __init__(self):
        pygame.init()
        pygame.mixer.init()
        self.sounda = pygame.mixer.Sound("chirp.wav")
    
    def on_destroy(self, hwnd, msg, wparam, lparam):
        pass

    def show_toast(self, title, text):
        os.system("""
                osascript -e 'display notification "{}" with title "{}"'
                """.format(text, title))

    def toast(self, subject, content):
        self.show_toast(subject, content)
        self.sounda.play()

In [8]:
config = {
    "ustc_id": "SA20011152",
    "ustc_pwd": "NTBNUY",
    "email_addr": "yjw1029@mail.ustc.edu.cn",
    "email_pwd": "@yjw56199801029",
}

In [9]:
class EPCBot(threading.Thread):

    # EPC网站相关url
    URL_ROOT     = "http://epc.ustc.edu.cn/"
    URL_LOGIN    = URL_ROOT + "n_left.asp"
    URL_BOOKED   = URL_ROOT + "record_book.asp"
    URL_BOOKABLE = URL_ROOT + "m_practice.asp?second_id=2002"
    
    def __init__(self, config:dict, ui=None):

        # 设置监听
        super(EPCBot, self).__init__(daemon=True)
        self.is_stopped = threading.Event()
        
        # 初始化相关参数
        self.ustc_id     = config["ustc_id"]
        self.ustc_pwd    = config["ustc_pwd"]
        self.email_addr  = config["email_addr"]
        self.email_pwd   = config["email_pwd"]

        # 初始化邮件通知类
        self.email_sender = EmailSender(self.email_addr, self.email_pwd)

        # 开启session会话
        self.session = requests.Session()
        self.session.headers.update({
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.75 Safari/537.36"
        })

    def login(self):
        data = {
            "submit_type": "user_login",
            "name": self.ustc_id,
            "pass": self.ustc_pwd,
            "user_type": "2",
            "Submit": "LOG IN"
        }
        resp = self.session.post(url=self.URL_LOGIN, data=data)
        if resp.status_code is 200 and not "登录失败" in resp.text:
            self.cookie = self.session.cookies.get_dict()
            print("Sucessfully Login")
            return True
        print("Fair to Login")
        return False

    def get_bookable_courses(self):
        bookable_epc = []
        data = {"querytype": "all"}
        resp = self.session.post(url=self.URL_BOOKABLE, data=data)
        html = BeautifulSoup(resp.text, "html.parser")
        if "登录后可以查看详细信息" in html.text:
            return "login"
        table = html.find_all("table")[4]
        tr = table.find_all("tr")
        form = html.find_all("form")
        for i in range(1, len(tr)):
            td = tr[i].find_all("td")
            date = td[5].get_text(separator=" ")
            date_, time_ = date.split(" ")
            tsp =  time.mktime(datetime.datetime.strptime(date_, "%Y/%m/%d").timetuple())
            bookable_epc.append({
                "unit": td[0].get_text(separator=" "),   # 预约单元
                "prof": td[3].get_text(separator=" "),   # 教师
                "hour": td[4].get_text(separator=" "),   # 学时
                "week": td[1].get_text(separator=" "),   # 教学周
                "wday": td[2].get_text(separator=" "),   # 星期
                "date": date,   # 上课时间
                "room": td[6].get_text(separator=" "),   # 上课教室
                "_url": form[i-1].get("action"),         # 表单链接
                "_new": True,                             # 是否为可预约课程
                "tsp": tsp
            })
        
        bookable_epc.sort(key=lambda x: x['tsp'])
        return bookable_epc
    
    def get_booked_epc(self):
        booked_epc = list()
        
        data = {"querytype": "new"}
        resp = self.session.post(url=self.URL_BOOKED, data=data)
        if resp.status_code is not 200: 
            self.print_log("Failed to fetch booked records.")
            return booked_epc, False

        html = BeautifulSoup(resp.text, "html.parser")
        table = html.find_all("table")[2]
        tr = table.find_all("tr")
        form = html.find_all("form")

        for i in range(1, len(tr)):
            td = tr[i].find_all("td")
            date = td[7].get_text(separator=" ")
            date_, time_ = date.split(" ")
            tsp =  time.mktime(datetime.datetime.strptime(date_, "%Y/%m/%d").timetuple())
            booked_epc.append({
                "unit": td[1].get_text(separator=" "),
                "prof": td[2].get_text(separator=" "),
                "hour": td[3].get_text(separator=" "),
                "week": td[5].get_text(separator=" "),
                "wday": td[6].get_text(separator=" "),
                "date": date,
                "room": td[8].get_text(separator=" "),
                "_url": form[i].get("action"),
                "_new": False,
                "tsp": tsp
            })
        booked_epc.sort(key=lambda x: x['tsp'])
        return booked_epc, True
    
    def submit_epc(self, epc_url, cmd):
        if cmd in ['cancel', 'submit']:
            return "Invalid CMD", False
        data = {"submit_type": "book_%s" % cmd}
        resp = self.session.post(url=self.URL_ROOT+epc_url, data=data)
        success = not "操作失败" in resp.text
        if success:
            return "Success", True
        else:
            return "Fail", False

In [11]:
epc_bot = EPCBot(config)
epc_bot.login()
booked_epc = epc_bot.get_booked_epc()
desktop_toaster = DesktopToaster()
min_booked_epc = booked_epc[0][-1]
min_time = min_booked_epc['tsp']
has_course = True
while True:
    bookable_course = epc_bot.get_bookable_courses()
    
    if bookable_course == "login":
        epc_bot.login()
        continue
        
    if len(bookable_course) == 0:
        content = f"无可预约课程"
        print(content)
        if has_course:
            desktop_toaster.toast("EPC Bot", content)
            has_course = False
        time.sleep(5 * 60)
        continue
        
    min_bookable = bookable_course[0]
    if min_bookable['tsp'] <= min_time:
        content = f"目前可选最早课程为{min_bookable['week']} {min_bookable['wday']} {min_bookable['date']}"
        print(content)
        desktop_toaster.toast("EPC Bot", content)
        rslt = input(f"是否要抢课? (y/n)")
        if rslt == "y":
            submit_flag = False
            while not submit_flag:
                if len(booked_epc) == 2:
                    rsp_cancel = epc_bot.submit_epc(min_booked_epc["_url"], "cancel")
                if rsp_cancel[-1]:
                    rsp_submit = epc_bot.submit_epc(min_bookable["_url"], "submit")
                submit_flag = rsp_submit[-1]
            content = f"成功抢课{min_bookable['week']} {min_bookable['wday']} {min_bookable['date']}"
            print(content)
            desktop_toaster.toast("EPC Bot", content)
        time.sleep(5 * 60)
    elif not has_course and min_bookabel['tsp'] > min_time:
        content = f"目前可选最早课程为{min_bookabel['week']} {min_bookabel['wday']} {min_bookabel['date']}"
        print(content)
        desktop_toaster.toast("EPC Bot", content)
        time.sleep(2 * 60)
    else:
        time.sleep(2 * 60)
    has_course = True

Sucessfully Login
无可预约课程
无可预约课程
无可预约课程
无可预约课程
无可预约课程
无可预约课程
无可预约课程


KeyboardInterrupt: 

In [54]:
# 取消课程
# data = {"second_id":"2002", "info_id":"2295", "week":"15", "week_day":"4", "book_sum_id":"113992"}
# epc_bot.session.post(url="http://epc.ustc.edu.cn/record_book.asp?", data=data)

1

In [20]:
epc_bot.get_booked_epc()

([{'unit': 'Marriage Customs',
   'prof': 'Edmund',
   'hour': '2',
   'week': '15',
   'wday': '周三',
   'date': '2021/6/16 14:00-15:30',
   'room': '-',
   '_url': 'record_book.asp?second_id=2002&info_id=1306&week=15&week_day=3&book_sum_id=114595',
   '_new': False,
   'tsp': 1623772800.0},
  {'unit': 'Chinese Cuisine',
   'prof': 'Jamie',
   'hour': '2',
   'week': '15',
   'wday': '周四',
   'date': '2021/6/17 14:00-15:30',
   'room': '-',
   '_url': 'record_book.asp?second_id=2002&info_id=2295&week=15&week_day=4&book_sum_id=113992',
   '_new': False,
   'tsp': 1623859200.0}],
 True)

In [21]:
input(f"是否要抢课?")

是否要抢课?yes


'yes'